In [1]:
import os 

In [2]:
%pwd

'/Users/sanahaidar/Desktop/NLP/text-summarizer/research'

In [3]:
os.chdir("../")

In [4]:
pwd

'/Users/sanahaidar/Desktop/NLP/text-summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path  
    unzip_dir: Path 


In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
 

In [26]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(self.config.artifacts_root)])
    
    # Data Ingestion Configuration
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([Path(config.root_dir)])
         
        return DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL, 
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [ ]:

import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [64]:
#Component for Data Ingestion
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config


    def download_file(self):

        if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL.strip(),   
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded! Info: \n{headers}")
    
        else:
            logger.error(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory 
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
           

In [65]:
#pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    logger.exception(e)
    raise e

[2025-07-26 12:44:57,685: INFO: common: YAML file config/config.yaml read successfully.]
[2025-07-26 12:44:57,688: INFO: common: YAML file params.yaml read successfully.]
[2025-07-26 12:44:57,689: INFO: common: Created directory: artifacts]
[2025-07-26 12:44:57,690: INFO: common: Created directory: artifacts/data_ingestion]
[2025-07-26 12:44:57,691: ERROR: 3867891498: File already exists of size: 8081 KB]
